In [52]:
import numpy as np
import os
import math
import cv2
import random

In [53]:
# assert x.shape  = (w,1)
# assert m.shape = (b,w)

In [54]:
b = 32
w = 21 * 21
alph = 10**-8
batch = 1
times = 10000
threshold = 10**-6
sigma = 0.45

In [55]:
def min_cost(x_, x_similar_, x_not_):
    assert x_.shape == x_similar_.shape == x_not_.shape
    (no_sense, num_sample) = x_.shape # 主要记录样本数目
#     print(b,w)
    m  = np.random.rand(b,w) 
    print(m)
    cost = 0
#     print('初始的Ｍ')
#     print(m)
    for t in range(times):
        # 获取一批数据
        batch_start = random.randint(0, num_sample-batch)
        
        x = x_[:, batch_start:batch_start + batch]
        x_similar = x_similar_[:, batch_start:batch + batch_start]
        x_not = x_not_[:, batch_start:batch_start + batch]
        
        # 计算前后两次误差，如果稳定在一个范围当中则停止计算
        cost_last = cost
        
        delt_w_0 = 2 * m
        
        delt_w_1 = (2 * np.dot(np.dot(m, (x - x_similar)), (x - x_similar).T) / batch - \
        2 *np.dot(np.dot(m, (x - x_not)), (x - x_not).T) / batch)
        
#         print('delt_w_1 is ' + str(delt_w_1))
#         print(delt_w_1.shape)
#         print(alph * delt_w_1)
        print('delt_w_0 is' )
        print(delt_w_0)
        print('        ')
        print(delt_w_0.shape)
        print('             ')
        print(sigma * delt_w_0)
        m = m -alph * delt_w_1 - sigma * delt_w_0
        tem_v = float(np.linalg.norm(np.dot(m, (x_ - x_similar_)),2) / np.sqrt(num_sample)) - float(np.linalg.norm(np.dot(m, (x_- x_not_)), 2) / np.sqrt(num_sample)) 
        cost = np.linalg.norm(m, 2) + tem_v
        print('m')
        print(m)
#         print('x-x_similar')
#         print(np.sum(x - x_similar))
#         print('x - x_not')
#         print(np.sum(x-x_not))
        tem_judge = float(np.linalg.norm(np.dot(m, (x - x_similar)),2))  - float(np.linalg.norm(np.dot(m, (x- x_not)), 2))
        print('该样本的损失为' + str(tem_judge))
        print('当前的损失为%f ' % (cost))
        print('前后两次的损失为%f' % (abs(cost - cost_last)))
        if abs(cost - cost_last) <= threshold:
            break
    return m

In [56]:
similar = list()# 构建每个patch块的相似词典
not_similar = list() # 构建每个patch快的非相似词典
x_patch = list() # 根据中心点构建patch块
patch_w = 21
patch_h = 21
not_patch_offset = int(patch_h / 2) 
photo_test_left =  os.path.join('photo', 'ref-4.bmp')
photo_test_right = os.path.join('photo', 'dst-4.bmp')
p_t_gray_l = cv2.imread(photo_test_left, 0) 
print(p_t_gray_l)
p_t_gray_r = cv2.imread(photo_test_right, 0) 
height, width = p_t_gray_l.shape
# print(p_t_gray_l.shape)
# print(p_t_gray_r.shape)
relate_point = np.loadtxt(os.path.join('data', 'duizhaoguanxi.txt'))
# print(relate_point)
right_points = relate_point[:,0:2]
# print('right_points is ' + str(right_points))
print(right_points.shape)
offset = relate_point[:, 2:4]
left_points = right_points + offset
right_points[:, [0,1]] = right_points[:, [1, 0]]
left_points[:, [0,1]] = left_points[:, [1, 0]]
for key, items in enumerate(right_points):
    try:
        # print(p_t_gray_r[int(items[0]) - 10 : int(items[0]) + 11 ,  int(items[1]) - 10 : int(items[1]) + 11].shape)
        # print(p_t_gray_l[int(left_points[key][0]) - 10 : int(left_points[key][0]) + 11 ,  int(left_points[key][1]) - 10 : int(left_points[key][1]) + 11].shape)
        # print( p_t_gray_l[int(left_points[key][0]) - 10 : int(left_points[key][0]) + 11 ,\
        #                                int(left_points[key][1]) + not_patch_offset - 10 : int(left_points[key][1]) + not_patch_offset + 11].shape)
        if p_t_gray_r[int(items[0]) - 10 : int(items[0]) + 11 ,  int(items[1]) - 10 : int(items[1]) + 11].shape == (21, 21) and \
        p_t_gray_l[int(left_points[key][0]) - 10 : int(left_points[key][0]) + 11 ,  int(left_points[key][1]) - 10 : int(left_points[key][1]) + 11].shape == (21, 21) and\
        p_t_gray_l[int(left_points[key][0]) - 10 : int(left_points[key][0]) + 11 ,\
                                        int(left_points[key][1]) + not_patch_offset - 10 : int(left_points[key][1]) + not_patch_offset + 11].shape == (21, 21):        
            x_patch.append(p_t_gray_r[int(items[0]) - 10 : int(items[0]) + 11 ,  int(items[1]) - 10 : int(items[1]) + 11].reshape(21*21))
            similar.append(p_t_gray_l[int(left_points[key][0]) - 10 : int(left_points[key][0]) + 11 ,  int(left_points[key][1]) - 10 : int(left_points[key][1]) + 11].reshape(21*21))
            not_similar.append(p_t_gray_l[int(left_points[key][0]) - 10 : int(left_points[key][0]) + 11 ,\
                                        int(left_points[key][1]) + not_patch_offset - 10 : int(left_points[key][1]) + not_patch_offset + 11].reshape(21*21))
    except:
        pass
num_patch = len(x_patch)
print(num_patch)
train_x_array = np.array(x_patch).reshape(num_patch, 21*21).T
train_x_similar_array = np.array(similar).reshape(num_patch, 21*21).T
train_x_not_similar_array = np.array(not_similar).reshape(num_patch, 21*21).T  
print('the shape of train_x_array is '+ str(train_x_array.shape))
print('the shape of train_x_similar_array is '+ str(train_x_not_similar_array.shape))
print('the shape of train_x_not_similar_array is '+ str(train_x_similar_array.shape))

[[14 14 14 ..., 13 13 13]
 [14 14 14 ..., 14 14 14]
 [14 14 14 ..., 15 14 14]
 ..., 
 [14 14 14 ..., 13 14 14]
 [15 15 14 ..., 14 13 13]
 [14 14 14 ..., 13 13 13]]
(5082, 2)
5061
the shape of train_x_array is (441, 5061)
the shape of train_x_similar_array is (441, 5061)
the shape of train_x_not_similar_array is (441, 5061)


In [57]:
m_best = min_cost(train_x_array, train_x_similar_array, train_x_not_similar_array)# 获取最好的m

[[ 0.05937846  0.6089586   0.99591049 ...,  0.67014884  0.7638735
   0.03218302]
 [ 0.57956889  0.20324749  0.21475305 ...,  0.20957615  0.00106038
   0.37310003]
 [ 0.524286    0.65444559  0.17699033 ...,  0.61329783  0.69879246
   0.68450629]
 ..., 
 [ 0.77650003  0.79278825  0.53294454 ...,  0.21456063  0.26390987
   0.46754226]
 [ 0.00279933  0.68556964  0.49911563 ...,  0.36914447  0.48231704
   0.91140898]
 [ 0.85452782  0.68820877  0.82039058 ...,  0.94107739  0.89858504
   0.1910649 ]]
delt_w_0 is
[[ 0.11875693  1.21791721  1.99182097 ...,  1.34029768  1.52774699
   0.06436603]
 [ 1.15913778  0.40649498  0.4295061  ...,  0.41915231  0.00212075
   0.74620007]
 [ 1.04857199  1.30889119  0.35398067 ...,  1.22659566  1.39758492
   1.36901258]
 ..., 
 [ 1.55300005  1.5855765   1.06588908 ...,  0.42912126  0.52781974
   0.93508452]
 [ 0.00559867  1.37113928  0.99823127 ...,  0.73828893  0.96463408
   1.82281795]
 [ 1.70905564  1.37641755  1.64078116 ...,  1.88215478  1.79717008
   0.

m
[[  1.24185638e-07   2.34100402e-07   4.04243350e-07 ...,  -2.70876389e-08
    4.02780942e-07   4.28324834e-07]
 [  1.17991001e-07   2.09832959e-07   3.62298106e-07 ...,  -2.85952022e-08
    3.60463375e-07   3.94923786e-07]
 [  1.23180963e-07   2.25750840e-07   3.80352224e-07 ...,  -2.65567975e-08
    3.86726342e-07   4.19690154e-07]
 ..., 
 [  1.40388793e-07   2.51965816e-07   4.27031676e-07 ...,  -3.30008177e-08
    4.26103300e-07   4.64219844e-07]
 [  1.19389339e-07   2.27330221e-07   3.88524674e-07 ...,  -2.91931881e-08
    3.89807680e-07   4.27428106e-07]
 [  1.30948010e-07   2.32636956e-07   3.98658554e-07 ...,  -2.35230041e-08
    3.99404877e-07   4.26201705e-07]]
该样本的损失为-0.123314061401
当前的损失为-0.019964 
前后两次的损失为0.099648
delt_w_0 is
[[  2.48371276e-07   4.68200805e-07   8.08486700e-07 ...,  -5.41752778e-08
    8.05561884e-07   8.56649669e-07]
 [  2.35982003e-07   4.19665917e-07   7.24596212e-07 ...,  -5.71904044e-08
    7.20926751e-07   7.89847571e-07]
 [  2.46361926e-07   4.51

m
[[  8.64975719e-12   7.13632180e-12   6.97764415e-12 ...,   1.03242506e-11
    8.75028697e-12   9.40083888e-12]
 [  7.90111508e-12   6.51131592e-12   6.36242133e-12 ...,   9.42212039e-12
    7.98224391e-12   8.58733248e-12]
 [  8.33153104e-12   6.87154823e-12   6.70882370e-12 ...,   9.94125202e-12
    8.42546312e-12   9.06029156e-12]
 ..., 
 [  9.24973994e-12   7.62776722e-12   7.45118260e-12 ...,   1.10311715e-11
    9.34754565e-12   1.00536872e-11]
 [  8.41073169e-12   6.93922671e-12   6.78223400e-12 ...,   1.00414047e-11
    8.51186170e-12   9.15747079e-12]
 [  8.54411853e-12   7.04480490e-12   6.88558584e-12 ...,   1.01973939e-11
    8.64167196e-12   9.28556507e-12]]
该样本的损失为-1.75653960579e-06
当前的损失为-0.000000 
前后两次的损失为0.000001


In [63]:
b = np.array([[1 , 2 , 3], [1 ,2 , 3], [1, 2, 3 ]])
print(b.shape)
for index , v in enumerate(b):
    if index == 0 :
        b = np.delete(b,index, 0)


(3, 3)


In [64]:
print b

[[1 2 3]
 [1 2 3]]
